In [1]:
## EXTRACTING FROM MULTIPLE PAGES


import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time



driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
base_url = "https://www.jumia.co.ke/flash-sales/?page={}"

product_list = []


for page_num in range (1, 9):
    url = base_url.format(page_num)
    driver.get(url)
    time.sleep(5)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    products = soup.find_all('article', class_='prd _fb _p col c-prd')
    
    if not products:
        print("No more products found.")
        
        break
    
    for product in products:
        name_tag = product.find('h3', class_='name')
        old_price = product.find('div', class_='old')
        new_price = product.find('div', class_='prc')
        dis = product.find('div', class_='bdg _dsct _sm')
        star_ratings_tag = product.find('div', class_='stars _s')
        items_remaining_tag = product.find('div', class_='stk')
        
        
        product_list.append({
            'Name':name_tag.text.strip() if name_tag else 'N/A',
            'Old_price':old_price.text.strip() if old_price else'N/A',
            'New_price':new_price.text.strip() if new_price else 'N/A',
            'Discount':dis.text.strip() if dis else 'N/A',
            'Star_ratings':star_ratings_tag.text.strip() if star_ratings_tag else 'N/A',
            'Items_remaining':items_remaining_tag.text.strip() if items_remaining_tag else 'N/A'

        })
    
driver.quit()

pl=pd.DataFrame(product_list)

pl.to_csv('/tmp/jumia_flash_sales.csv')


No more products found.


,Name,Old_price,New_price,Discount,Star_ratings,Items_remaining
0,Ramtons RF/257- 2 Door Direct Cool Fridge - 21...,"KSh 48,900","KSh 33,176",32%,4.1 out of 5,41 items left
1,Simple Invisible Antiperspirant Roll On 45ml,KSh 399,KSh 320,20%,4.6 out of 5,89 items left
2,Brown Orchid Perfume For Men EDP +deo Spray,"KSh 2,482","KSh 1,699",32%,4.5 out of 5,93 items left
3,Spiderman Watch LED Light Kid's Watch For Boys...,KSh 430,KSh 271,37%,N/A,20 items left
4,Istanbul 2 Modern TV Stand With LED Lights,"KSh 6,599","KSh 5,250",20%,4.1 out of 5,100 items left
...,...,...,...,...,...,...
143,8 Tier Stackable Shoe Rack - Black,"KSh 2,412","KSh 2,100",13%,N/A,24 items left
144,"Infinix Smart 10, 6.67"", 3GB RAM + 64GB (Dual ...","KSh 10,900","KSh 9,070",17%,N/A,59 items left
145,6-piece luxury fashion square women's watch br...,"KSh 1,901",KSh 459,76%,N/A,28 items left
146,Waterproof Bedspread Mattress Protectors-120*2...,"KSh 3,004 - KSh 3,204","KSh 2,090 - KSh 2,403",30%,N/A,100 items left


In [2]:
##TRANSFORM THE DATA


In [3]:
#removing the prefix and suffix

pl = pd.read_csv('/tmp/jumia_flash_sales.csv')


pl['Old_price'] = pl['Old_price'].str.replace('KSh','').str.strip()

pl['New_price']=pl['New_price'].str.replace('KSh', '').str.strip()

pl['Discount'] = pl['Discount'].str.replace('%', '').str.strip()

pl['Star_ratings'] = pl['Star_ratings'].str.replace('out of 5', '').str.strip()

pl.rename(columns={'Star_ratings':'Star_ratings_out_of_5'}, inplace=True)

pl['Old_price']=pl['Old_price'].str.replace(',', '').str.strip()

pl['New_price']=pl['New_price'].str.replace(',', '').str.strip()


##remove N/A

pl['Old_price']=pl['Old_price'].str.replace('N/A', '0').str.strip()

pl['New_price']=pl['New_price'].str.replace('N/A', '0').str.strip()

pl['Discount']=pl['Discount'].str.replace('N/A', '0').str.strip()

pl.Discount.fillna('0', inplace=True)

pl.Old_price.fillna('0', inplace=True)

#change datatypes

pl['Discount']=pl['Discount'].astype(int)

pl['Discount']=pl['Discount']/100

pl.rename(columns= {'Discount':'Discount(/100)'}, inplace=True)


#removing unique figures

pl['Old_price'] = pl['Old_price'].str.split('-').str[-1].str.replace(',', '').str.strip()

pl['Old_price'] = pl['Old_price'].astype(int)

pl['New_price'] = pl['New_price'].str.split('-').str[-1].str.replace(',', '').str.strip()

pl['New_price']=pl['New_price'].astype(int)


In [4]:
pl

,Unnamed: 0,Name,Old_price,New_price,Discount(/100),Star_ratings_out_of_5,Items_remaining
0,0,Ramtons RF/257- 2 Door Direct Cool Fridge - 21...,48900,33176,0.32,4.1,41 items left
1,1,Simple Invisible Antiperspirant Roll On 45ml,399,320,0.20,4.6,89 items left
2,2,Brown Orchid Perfume For Men EDP +deo Spray,2482,1699,0.32,4.5,93 items left
3,3,Spiderman Watch LED Light Kid's Watch For Boys...,430,271,0.37,NaN,20 items left
4,4,Istanbul 2 Modern TV Stand With LED Lights,6599,5250,0.20,4.1,100 items left
...,...,...,...,...,...,...,...
143,143,8 Tier Stackable Shoe Rack - Black,2412,2100,0.13,NaN,24 items left
144,144,"Infinix Smart 10, 6.67"", 3GB RAM + 64GB (Dual ...",10900,9070,0.17,NaN,59 items left
145,145,6-piece luxury fashion square women's watch br...,1901,459,0.76,NaN,28 items left
146,146,Waterproof Bedspread Mattress Protectors-120*2...,3204,2403,0.30,NaN,100 items left


In [5]:
#save
pl.to_csv('Jumia_flash_sales')

In [6]:
##LOAD THE DATA